# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [80]:
# import libraries
import pandas as pd
import nltk
from sqlalchemy import create_engine
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import re

In [48]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/leo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/leo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/leo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/leo/nltk_data...


In [50]:
# load data from database
engine = create_engine('sqlite:///message.db')
df = pd.read_sql_table('message',engine)
X = df['message']
y = df.drop(['id','message','original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [51]:
def tokenize(text):
    # normalize case and remove punct
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenize text
    tokens = word_tokenize(text)
    # lemmatize and remove stop words
    tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens if w not in stopwords.words("english")]
    return tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [52]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('moc', MultiOutputClassifier(KNeighborsClassifier())),
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [54]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [55]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9fa2b95310>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [81]:
y_pred = pipeline.predict(X_test)

In [88]:
for i in range(len(y_test.columns)):
    col = y_test.columns[i]
    print('************* '+col+' ************')
    print(classification_report(y_test[col], y_pred[:,i]))
    print(precision_recall_fscore_support(y_test[col], y_pred[:,i], average='macro'))
    

************* related ************
              precision    recall  f1-score   support

         0.0       0.39      0.12      0.18       841
         1.0       0.67      0.91      0.77      1687
         2.0       0.50      0.04      0.07        28

    accuracy                           0.64      2556
   macro avg       0.52      0.35      0.34      2556
weighted avg       0.57      0.64      0.57      2556

************* request ************
              precision    recall  f1-score   support

         0.0       0.65      0.89      0.75      1640
         1.0       0.42      0.14      0.21       916

    accuracy                           0.62      2556
   macro avg       0.54      0.52      0.48      2556
weighted avg       0.57      0.62      0.56      2556

************* offer ************
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2553
         1.0       0.00      0.00      0.00         3

    accuracy             

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      2523
         1.0       0.00      0.00      0.00        33

    accuracy                           0.99      2556
   macro avg       0.49      0.50      0.50      2556
weighted avg       0.97      0.99      0.98      2556

************* missing_people ************
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      2532
         1.0       0.00      0.00      0.00        24

    accuracy                           0.99      2556
   macro avg       0.50      0.50      0.50      2556
weighted avg       0.98      0.99      0.99      2556

************* refugees ************
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      2521
         1.0       0.00      0.00      0.00        35

    accuracy                           0.99      2556
   macro avg       0.49      0.50      0.50      2

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.